In [1]:
import numpy as np

In [2]:
# old base vectors in old frame
e1 = np.array([1., 0.])
e2 = np.array([0., 1.])

# new base vectors in old frame
E1 = np.array([0., 1.])
E2 = np.array([-1., 0.])

# direction cosine matrix
DCM = np.stack([E1, E2])

# the transpose of DCM transforms the base vectors as column arrays
assert np.all(np.isclose(DCM.T @ e1, E1, rtol=1e-05, atol=1e-08))
assert np.all(np.isclose(DCM.T @ e2, E2, rtol=1e-05, atol=1e-08))

# the DCM transforms the base vectors as row arrays
assert np.all(np.isclose(e1 @ DCM, E1, rtol=1e-05, atol=1e-08))
assert np.all(np.isclose(e2 @ DCM, E2, rtol=1e-05, atol=1e-08))

# transform the complete frame at once
assert np.all(np.isclose(DCM @ np.stack([e1, e2]), np.stack([E1, E2]), 
                         rtol=1e-05, atol=1e-08))

In [3]:
DCM

array([[ 0.,  1.],
       [-1.,  0.]])

In [4]:
from sympy.physics.vector import ReferenceFrame

In [5]:
source = ReferenceFrame('source')
target = source.orientnew('target', 'Body', [0, 0, 90*np.pi/180],  'XYZ')
DCM = np.array(target.dcm(source).evalf()).astype(float)[:2, :2]
DCM[np.abs(DCM) < 1e-12] = 0.

In [6]:
DCM

array([[ 0.,  1.],
       [-1.,  0.]])

In [7]:
from sigmaepsilon.math.linalg import Vector, ReferenceFrame

In [8]:
A = ReferenceFrame(name='A', axes=np.eye(3))
B = A.orient_new('Body', [0, 0, 90*np.pi/180], 'XYZ', name='B')
B.dcm()

ArrayBase([[ 6.123234e-17,  1.000000e+00,  0.000000e+00],
           [-1.000000e+00,  6.123234e-17,  0.000000e+00],
           [ 0.000000e+00,  0.000000e+00,  1.000000e+00]])

In [9]:
v = Vector([0.0, 1.0, 0.0], frame=A)
v.view(B)

VectorBase([0., 1., 0.])

In [10]:
v.orient('Body', [0, 0, -90*np.pi/180], 'XYZ')
v.view(B)

VectorBase([0., 1., 0.])

In [11]:
v.view()

VectorBase([0., 1., 0.])

In [12]:
# the original frame    
A = ReferenceFrame(dim=3, axes=np.eye(3))

# the original vector
vA = Vector([1., 0., 0.], frame=A)

# random rotation
amounts = [0., 0., 0.]
amounts[2] = 90 * np.pi / 180
B = A.orient_new('Body', amounts, 'XYZ')

In [13]:
vA.show()

array([1., 0., 0.])

In [14]:
vA.show(A)

array([1., 0., 0.])

In [15]:
vA.show(B)

array([ 6.123234e-17, -1.000000e+00,  0.000000e+00])

In [16]:
vA2 = Vector(vA.show(B), frame=B)

In [17]:
vA2.show(A)

array([1., 0., 0.])

In [18]:
vA2.show()

array([1., 0., 0.])

In [19]:
vA2.array

VectorBase([ 6.123234e-17, -1.000000e+00,  0.000000e+00])

In [20]:
vA2.frame

ReferenceFrame(ArrayBase([[ 6.123234e-17,  1.000000e+00,  0.000000e+00],
           [-1.000000e+00,  6.123234e-17,  0.000000e+00],
           [ 0.000000e+00,  0.000000e+00,  1.000000e+00]]))

In [21]:
vA2.show()

array([1., 0., 0.])

In [1]:
import unittest
import doctest
import operator

import numpy as np
from numpy import vstack
from scipy.sparse import csr_matrix as csr_scipy
from numba import njit
import sympy as sy

from sigmaepsilon.math.linalg.utils import random_pos_semidef_matrix, random_posdef_matrix
from sigmaepsilon.math.logical import ispossemidef, isposdef
from sigmaepsilon.math.linalg import (ReferenceFrame, RectangularFrame, CartesianFrame, ArrayWrapper,
                            Vector, Tensor, Tensor2, Tensor4, Tensor2x3, Tensor4x3,
                            inv3x3, det3x3, inv2x2u, inv3x3u, inv2x2, det2x2)
from sigmaepsilon.math.linalg.solve import linsolve, reduce
from sigmaepsilon.math.linalg.sparse import JaggedArray, csr_matrix
from sigmaepsilon.math.linalg.sparse.jaggedarray import get_data
import sigmaepsilon.math.linalg.sparse.utils as spu
from sigmaepsilon.math import linalg
from sigmaepsilon.math.linalg.exceptions import (LinalgOperationInputError, LinalgMissingInputError, 
                                       LinalgInvalidTensorOperationError, TensorShapeMismatchError)
from sigmaepsilon.math.linalg import utils as lautils
from sigmaepsilon.math import repeat
from sigmaepsilon.math.linalg import top, dot, cross

In [2]:
i=1
a=120.

# the original frame
A = ReferenceFrame(dim=3)

# the original vector
vA = Vector([1., 0., 0.], frame=A)

# random rotation
amounts = [0., 0., 0.]
amounts[i] = a * np.pi / 180
B = A.orient_new('Body', amounts, 'XYZ')

# Image of vA in B
vB = Vector(vA.show(B), frame=B)

# test if the image of vB in A is the same as vA
assert np.all(np.isclose(vB.show(A), vA.array))